This solves a basic CSP (locations only, no NLU)
Current test data: Monday 08 May 2017 crossword

In [1]:
#Imports
!pip install python-constraint
from constraint import Problem
import csv
from gensim.models import Word2Vec
import gensim.downloader as api
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/saahil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#Global variables
wordlist = []

In [3]:
def printGrid(grid):
    for row in grid:
        for col in row:
            print(col,end=" ")
        print()

In [4]:
def findStartingCells(grid):
    #Output format: list of lists of format [startrow,startcol,wordlen,isAcross]
    row_clues = []
    col_clues = []

    for row_index, row in enumerate(grid):
        col_index = 0
        while col_index < len(row):
            if row[col_index] != '.':
                start_col_index = col_index
                length = 0
                while col_index < len(row) and row[col_index] != '.':
                    length += 1
                    col_index += 1
                row_clues.append([row_index,start_col_index,length,True])
            col_index += 1

    num_cols = len(grid[0])
    for col_index in range(num_cols):
        row_index = 0
        while row_index < len(grid):
            if grid[row_index][col_index] != '.':
                start_row_index = row_index
                length = 0
                while row_index < len(grid) and grid[row_index][col_index] != '.':
                    length += 1
                    row_index += 1
                col_clues.append([start_row_index,col_index,length,False])
            row_index += 1
    return row_clues + col_clues

In [5]:
def findIntersections(clues):
    intersections = []
    for i,clue1 in enumerate(clues):
        for j,clue2 in enumerate(clues):
            if i < j:
                row1,col1,len1,isAcross1 = clue1
                row2,col2,len2,isAcross2 = clue2

                #check intersection
                if isAcross1 == True and isAcross2 == False:                    
                    if row2 <= row1 < row2+len2 and col1 <= col2 < col1+len1:
                        #clue1 is across - subtr cols
                        posInClue1 = col2 - col1
                        posInClue2 = row1 - row2
                        intersections.append(((i,posInClue1),(j,posInClue2)))
                elif isAcross1 == False and isAcross2 == True:
                    if row1 <= row2 < row1+len1 and col2 <= col1 < col2+len2:
                        #clue1 is down - subtr rows
                        posInClue1 = row2 - row1
                        posInClue2 = col1 - col2
                        intersections.append(((i,posInClue1),(j,posInClue2)))
    return intersections

In [6]:
def importWordList(fp):
    with open(fp, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            toAdd = row[0].replace(" ","").upper()
            wordlist.append(toAdd)    

In [7]:
def createCSP(wordlist,clues):
    problem = Problem()

    #add a variable for each clue
    for i, (row,col,length,isAcross) in enumerate(clues):
        domain = [word for word in wordlist if len(word)==length]
        problem.addVariable(i,domain)

    intersections = findIntersections(clues)
    for (clue1,pos1),(clue2,pos2) in intersections:
        problem.addConstraint(lambda w1, w2, p1=pos1, p2=pos2: w1[p1] == w2[p2], (clue1, clue2))


    return problem

In [8]:
def solveCSP(problem):
    solutions = problem.getSolutions()
    print("Num solutions found:",len(solutions))

    return solutions

In [9]:
def fillInGrid(emptyGrid,clues,solution):
    newGrid = emptyGrid.copy()
    for clue,word in solution.items():
        row,col,length,isAcross = clues[clue]
        if isAcross:
            for i in range(length):
                newGrid[row][col+i] = word[i]
        else:
            for i in range(length):
                newGrid[row+i][col] = word[i]
    return newGrid

In [10]:
#Import full wordlist
wordlist = []
importWordList("answers_only.csv") #stored in var wordlist

In [11]:
#Define test puzzle
solutionGrid = [['G', 'P', 'A', '.', 'M', 'A', 'G', 'O', 'O', '.', 'S', 'C', 'O', 'W', 'S'], ['O', 'L', 'D', '.', 'C', 'L', 'O', 'T', 'H', '.', 'I', 'O', 'N', 'I', 'A'], ['B', 'A', 'D', '.', 'E', 'L', 'L', 'E', 'F', 'A', 'N', 'N', 'I', 'N', 'G'], ['I', 'N', 'L', 'A', 'W', '.', '.', 'R', 'U', 'N', '.', 'C', 'O', 'D', 'E'], ['G', 'E', 'E', 'Y', 'A', 'T', 'H', 'I', 'N', 'K', '.', 'E', 'N', 'C', '.'], ['.', '.', '.', 'E', 'N', 'Y', 'A', '.', '.', 'H', 'A', 'R', 'S', 'H', '.'], ['A', 'N', 'T', 'S', '.', 'P', 'R', 'O', 'M', '.', 'I', 'N', 'O', 'I', 'L'], ['S', 'E', 'E', '.', 'B', 'E', 'E', 'B', 'A', 'L', 'M', '.', 'U', 'M', 'A'], ['K', 'I', 'T', 'T', 'Y', '.', 'S', 'I', 'L', 'O', '.', 'A', 'P', 'E', 'X'], ['.', 'L', 'E', 'A', 'S', 'E', '.', '.', 'I', 'N', 'S', 'T', '.', '.', '.'], ['.', 'Y', 'A', 'K', '.', 'T', 'E', 'A', 'K', 'E', 'T', 'T', 'L', 'E', 'S'], ['N', 'O', 'T', 'E', '.', 'T', 'L', 'C', '.', '.', 'A', 'N', 'G', 'L', 'E'], ['C', 'U', 'E', 'T', 'H', 'E', 'M', 'U', 'S', 'I', 'C', '.', 'B', 'I', 'C'], ['A', 'N', 'T', 'E', 'S', '.', 'S', 'T', 'A', 'R', 'K', '.', 'T', 'O', 'T'], ['A', 'G', 'E', 'N', 'T', '.', 'T', 'E', 'X', 'A', 'S', '.', 'Q', 'T', 'S']]
emptyGrid = [['.' if cell == '.' else ' ' for cell in row] for row in solutionGrid]
answersList = ["GPA","MAGOO","SCOWS","OLD","CLOTH","IONIA","BAD","ELLEFANNING","INLAW","RUN","CODE","GEEYATHINK","ENC","ENYA","HARSH","ANTS","PROM","INOIL","SEE","BEEBALM","UMA","KITTY","SILO","APEX","LEASE","INST","YAK","TEAKETTLES","NOTE","TLC","ANGLE","CUETHEMUSIC","BIC","ANTES","STARK","TOT","AGENT","TEXAS","QTS","GOBIG","PLANE","ADDLE","MCEWAN","ALL","GOL","OTERI","OHFUN","SIN","CONCERN","ONIONSOUP","WINDCHIME","SAGE","ANKH","AYES","TYPE","HARES","AIM","ASK","NEILYOUNG","TETEATETE","OBI","MALIK","LAX","BYS","LONE","TAKETEN","ATTN","ETTE","STACKS","ELMST","ACUTE","LGBTQ","ELIOT","SECTS","NCAA","HST","SAX","IRA"]

In [12]:
#Create and solve CSP
clues = findStartingCells(solutionGrid)
problem = createCSP(answersList,clues)
solutions = solveCSP(problem)

#Print generated solutions
print("-----------------------------------------------------------------------------------------------------------------------")
for i,solution in enumerate(solutions):
    print()
    print(f"Solution #{i+1}")
    printGrid(fillInGrid(emptyGrid,clues,solution))

print()
print("Actual Solution")
printGrid(solutionGrid)
print()

#Compute and print correctness
#Correctness is defined as the average number of correct cells across all generated solutions
#Note that in a 15x15 setup the core is small, so we expect there to be usually only 0 or 1 solution generated; averaging thus usually won't apply

#TODO: If a solution isn't fully generated, we don't get an output and our accuracy is 0. How do we allow for partial solutions (i.e. CSP partially solved and then leads to contradiction)?
points_possible = 225 - sum(row.count('.') for row in solutionGrid)
correctness = 0
for solution in solutions:
    score = points_possible
    candidateSoln = fillInGrid(emptyGrid,clues,solution)
    for i in range(15):
        for j in range(15):
            if candidateSoln[i][j] != solutionGrid[i][j]:
                score -= 1
    correctness += score/points_possible
if len(solutions) != 0:
    correctness /= len(solutions)

print(f"Average accuracy: {round(correctness,3)} on {len(solutions)} solution(s) generated")
print("-----------------------------------------------------------------------------------------------------------------------")


Num solutions found: 1
-----------------------------------------------------------------------------------------------------------------------

Solution #1
G P A . M A G O O . S C O W S 
O L D . C L O T H . I O N I A 
B A D . E L L E F A N N I N G 
I N L A W . . R U N . C O D E 
G E E Y A T H I N K . E N C . 
. . . E N Y A . . H A R S H . 
A N T S . P R O M . I N O I L 
S E E . B E E B A L M . U M A 
K I T T Y . S I L O . A P E X 
. L E A S E . . I N S T . . . 
. Y A K . T E A K E T T L E S 
N O T E . T L C . . A N G L E 
C U E T H E M U S I C . B I C 
A N T E S . S T A R K . T O T 
A G E N T . T E X A S . Q T S 

Actual Solution
G P A . M A G O O . S C O W S 
O L D . C L O T H . I O N I A 
B A D . E L L E F A N N I N G 
I N L A W . . R U N . C O D E 
G E E Y A T H I N K . E N C . 
. . . E N Y A . . H A R S H . 
A N T S . P R O M . I N O I L 
S E E . B E E B A L M . U M A 
K I T T Y . S I L O . A P E X 
. L E A S E . . I N S T . . . 
. Y A K . T E A K E T T L E S 
N O T E . T L C . . A

In [13]:
emptyGrid = [['■' if cell == '.' else cell for cell in row] for row in solutionGrid]
printGrid(emptyGrid)
print(clues)
print(findIntersections(clues))

G P A ■ M A G O O ■ S C O W S 
O L D ■ C L O T H ■ I O N I A 
B A D ■ E L L E F A N N I N G 
I N L A W ■ ■ R U N ■ C O D E 
G E E Y A T H I N K ■ E N C ■ 
■ ■ ■ E N Y A ■ ■ H A R S H ■ 
A N T S ■ P R O M ■ I N O I L 
S E E ■ B E E B A L M ■ U M A 
K I T T Y ■ S I L O ■ A P E X 
■ L E A S E ■ ■ I N S T ■ ■ ■ 
■ Y A K ■ T E A K E T T L E S 
N O T E ■ T L C ■ ■ A N G L E 
C U E T H E M U S I C ■ B I C 
A N T E S ■ S T A R K ■ T O T 
A G E N T ■ T E X A S ■ Q T S 
[[0, 0, 3, True], [0, 4, 5, True], [0, 10, 5, True], [1, 0, 3, True], [1, 4, 5, True], [1, 10, 5, True], [2, 0, 3, True], [2, 4, 11, True], [3, 0, 5, True], [3, 7, 3, True], [3, 11, 4, True], [4, 0, 10, True], [4, 11, 3, True], [5, 3, 4, True], [5, 9, 5, True], [6, 0, 4, True], [6, 5, 4, True], [6, 10, 5, True], [7, 0, 3, True], [7, 4, 7, True], [7, 12, 3, True], [8, 0, 5, True], [8, 6, 4, True], [8, 11, 4, True], [9, 1, 5, True], [9, 8, 4, True], [10, 1, 3, True], [10, 5, 10, True], [11, 0, 4, True], [11, 5, 3, True], [11, 10, 5